# Attempt at writing down generalized solver
1. Have written up a library of functions. Testing here to write down a single function to call to get us everything we want. 

In [1]:
from broyden import *
from solver_helper import *

import matplotlib.pyplot as plt 
import time

n0 = 0.153

In [7]:
import pandas as pd

### Goal: 
1. Declare our set of baryons, leptons, and mesons. 
2. In the context of our gm3 model: get all the equations. 
3. Solve. 
4. Print out a data table. 

In [2]:
# have to initialize eos
gm3 = eos(g_sigma_N = 8.784820, g_omega_N = 8.720086, g_rho_N = 8.544795, g_phi_N = 0.0, b = 0.008628, c = -0.002433,\
             g_sigma_H = 5.408849, g_omega_H = 5.813391, g_rho_H = 0.0, g_phi_H = -4.110688)
#gm3_sym = eos(g_sigma_N_sym, g_omega_N_sym, g_rho_N_sym, g_phi_N_sym, b_sym, c_sym, g_sigma_H_sym, g_omega_H_sym, g_rho_H_sym, g_phi_H_sym)

# can think about writing a symbolic declarerer func since this is a little ugly
# - can store values ahead of time... but would be good to be

In [3]:
#def load_model(eos_array):
    # pass a Walecka RMF type model with coupling constants and stores coupling constants in both
    # an eos object and distributes values to the baryons 
        

In [2]:
# have to declare lists
# note: baryons are capitalized! for reasons... 
baryon_list = [Proton, Neutron]
meson_list = [sigma, omega, rho]
lepton_list = [electron, muon]

In [5]:
# have to initialize baryons with coupling constants
# and sigma meson with sigma meson coupling 
init(gm3, baryon_list)

In [6]:
eqn1 = sigma_eom(baryon_list)
eqn1
# at this point, we see that we have a purely symbolic function. We deal with this by then substituting in numerical values.
# alternatively, we can substitute in directly some numerical values from the git go. Which would probably be more efficient 
# but sacrifice some clarity. 

b*g_sigma_N**3*m_n*sigma**2 + c*g_sigma_N**4*sigma**3 - 1.0*g_sigma_N**2*(-g_sigma_N*sigma + m_n)*(k_n*sqrt(k_n**2 + (-g_sigma_N*sigma + m_n)**2) - (-g_sigma_N*sigma + m_n)**2*log(sqrt((k_n + sqrt(k_n**2 + (-g_sigma_N*sigma + m_n)**2))**2/(-g_sigma_N*sigma + m_n)**2)))/pi**2 - 1.0*g_sigma_N**2*(-g_sigma_N*sigma + m_p)*(k_p*sqrt(k_p**2 + (-g_sigma_N*sigma + m_p)**2) - (-g_sigma_N*sigma + m_p)**2*log(sqrt((k_p + sqrt(k_p**2 + (-g_sigma_N*sigma + m_p)**2))**2/(-g_sigma_N*sigma + m_p)**2)))/pi**2 + m_sigma**2*sigma

In [7]:
eqn2 = omega_eom(baryon_list)
eqn2

-g_omega_N*k_n**3 - g_omega_N*k_p**3 + 3*m_omega**2*omega*pi**2

In [8]:
eqn3 = rho_eom(baryon_list)
eqn3

0.5*g_rho_N*k_n**3 - 0.5*g_rho_N*k_p**3 + 3*m_rho**2*pi**2*rho

In [9]:
eqn4 = phi_eom(baryon_list)
eqn4

-g_phi_N*k_n**3 - g_phi_N*k_p**3 + 3*m_phi**2*phi*pi**2

In [10]:
eqn5 = beta_equilibrium(baryon_list)
eqn5[0]

1.0*g_rho_N*rho + sqrt(k_e**2 + m_e**2) - sqrt(k_n**2 + (-g_sigma_N*sigma + m_n)**2) + sqrt(k_p**2 + (-g_sigma_N*sigma + m_p)**2)

In [11]:
eqn6 = charge_conservation(baryon_list, lepton_list)
eqn6

-k_e - k_mu + k_p

In [11]:
func_gen = [sigma_eom, omega_eom, rho_eom, phi_eom, beta_equilibrium, charge_conservation, baryon_num_conservation] 

# need to figure out how to generate all these equations and like unpack the beta equilibrium ones.. 
def sys_eqn_gen(baryon_sym_list, lepton_sym_list):
    # function to generate all our equations and to store in an array
    # called sys_eqn_gen 
    
    sys_eqn = []
    
    for function in func_gen:
        if (function == charge_conservation):
            # since charge conservation depends on both baryons and leptons
            sys_eqn.append(function(baryon_sym_list, lepton_sym_list))
        elif (function == beta_equilibrium):
             # beta condition function returns an array with multiple equations
            # we unpack that here
            beta_conditions = function(baryon_sym_list)
            for equation in beta_conditions:
                sys_eqn.append(equation)
        else:
            sys_eqn.append(function(baryon_sym_list))
            
    return sys_eqn

In [12]:
A = sys_eqn_gen(baryon_list, lepton_list)

In [15]:
def subs(sys_eqn, baryon_list, meson_list, lepton_list):
    for i in range(len(sys_eqn)):
        sys_eqn[i] = substitution(sys_eqn[i], baryon_list, meson_list, lepton_list)
    return sys_eqn

In [16]:
subs(A)

[-14.4901732481197*sigma**3 + 5492.55393864515*sigma**2 + 302500.0*sigma - 7.8192660309548*(939.0 - 8.78482*sigma)*(939.0*k_n*sqrt(1.13414560841808e-6*k_n**2 + (1 - 0.00935550585729499*sigma)**2) - 881721.0*(1 - 0.00935550585729499*sigma)**2*log(1.0*sqrt((0.00106496272630458*k_n + sqrt(1.13414560841808e-6*k_n**2 + (1 - 0.00935550585729499*sigma)**2))**2/(1 - 0.00935550585729499*sigma)**2))) - 7.8192660309548*(939.0 - 8.78482*sigma)*(939.0*k_p*sqrt(1.13414560841808e-6*k_p**2 + (1 - 0.009355505857295*sigma)**2) - 881721.0*(1 - 0.009355505857295*sigma)**2*log(1.0*sqrt((0.00106496272630458*k_p + sqrt(1.13414560841808e-6*k_p**2 + (1 - 0.009355505857295*sigma)**2))**2/(1 - 0.009355505857295*sigma)**2))),
 -8.720086*k_n**3 - 8.720086*k_p**3 + 18152837.6779784*omega,
 4.2723975*k_n**3 - 4.2723975*k_p**3 + 17555065.3482176*rho,
 29.6088132032681*m_phi**2*phi,
 8.544795*rho + sqrt(k_e**2 + 0.2601) - 939.0*sqrt(1.13414560841808e-6*k_n**2 + (1 - 0.00935550585729499*sigma)**2) + 939.0*sqrt(1.134145

In [18]:
A[6]

k_n**3 + k_p**3 + 29.6088132032681*n_B

In [ ]:
def solver(baryon_list, meson_list, lepton_list, nb_array = 0.0):
    
    # generate nB array
    nb_array = np.arange(0.27, 0.5, 0.01)
    nb_array_mev = nb_array*n0*hc**3
    
    # generate data storage arrays
    data = pd.DataFrame([], columns = column_name(baryon_list, meson_list, lepton_list) )
    data_list = []
    
    #for loop to iterate through nB, solve and store and update 
    for i in range(len(nb_array_mev)):
        
    
    # provide an initial guess
    
    
    # return tabular data for each array. 

In [3]:
def column_name(baryon_list, meson_list, lepton_list):
    # generate column names
    columns = ['nB/n0']
    
    for meson in meson_list:
        columns.append(meson.name + " " + 'field (MeV)')
        
    for baryon in baryon_list:
        columns.append(baryon.name + " " + 'kF (MeV)')
        columns.append(baryon.name + " " + 'frac')
        columns.append(baryon.name + " " + 'chem pot (MeV)')
        
    for lepton in lepton_list:
        columns.append(lepton.name + " " + "kF (MeV)")
        columns.append(lepton.name + " " + "frac")
        columns.append(lepton.name + " " + "chem pot (MeV)")
    
    
    return columns

In [5]:
A = column_name(baryon_list, meson_list, lepton_list)

In [9]:
data = pd.DataFrame([], columns = A)
data

,nB/n0,sigma field (MeV),omega field (MeV),rho field (MeV),Proton kF (MeV),Proton frac,Proton chem pot (MeV),Neutron kF (MeV),Neutron frac,Neutron chem pot (MeV),electron kF (MeV),electron frac,electron chem pot (MeV),muon kF (MeV),muon frac,muon chem pot (MeV)


In [ ]:
# note: appending to data frame at each iteration will be intensive
# will be better to work with lists and then contentacate the list to the dataframe at the end 

In [ ]:
data_list = []

In [21]:
nb_array = np.arange(0.27, 0.5, 0.01)
nb_array_mev = nb_array*n0*hc**3
data = pd.DataFrame()

In [ ]:
def fraction():
    # take data from previous section and then find the fraction... 

In [ ]:
def new_solver():
    # load in entire system of particles to consider
    
    # need to build in some method of updating the system of equations to solve as chemical potential becomes high
    # enough for new particles to begin populating the system... 
    
    # 
    
    # 

### Threshold Condition
We have a condition on the baryons. They will begin to occupy the system when their chemical potential satisfies the following equation
$$
    \mu_n - q\mu_e \geq g_{\omega B}\omega + I_{3B}g_{\rho B}\rho + m_* \qquad m_* = m_B - g_\sigma \sigma
$$
which is derived from the the chemical potential of the baryon being greater than or equal to its chemical potential in the case where it's momentum is zero. Ie, 
$$
     {\mu_B}_{\lim k_F \rightarrow 0} = \lim_{k_F \rightarrow 0}\left[\sqrt{k_F^2 + m_*^2} + g_{\omega B}\omega + I_{3B}g_{\rho B}\rho\right] 
$$
At each step, we need to check if the chemical potentials have become significant enough to populate. Adds significant complexity to our system tbh.

In [32]:
def bare_chemical_potential(baryon, omega_field, rho_field, sigma_field):
    # calculates bare chemical potential 
    return baryon.num_g_omega * omega_field + baryon.isospin * barygon.g_rho * rho_field\
            + baryon.mass - baryon.g_sigma * sigma_field

In [ ]:
def threshold(baryon, omega_field, rho_field, sigma_field, neutron_chem_pot, electron_chem_pot):
    # checks at end of each step if the chemical potential is high enough to begin populating 
    # a new baryon species 
    # would then need to update 
    if (neutron_chem_pot - baryon.charge * electron_chem_pot >= bare_chemical_potential(baryon, omega_field, rho_field, sigma_field)):
        baryon_list.append(baryon)
        return True
    elif
        return False

In [ ]:
def mean_field_solve(baryon_list, lepton_list, meson_list):
    # Takes list of particles and returns the solved values
    # stores in array 
    
    # begin with npe matter at the beginning
    current_baryons = [Proton, Neutron]
    current_leptons = [electron]
    
    # store other baryons, leptons in these lists
    # baryon list is sorted in order of mass and hence order of appearance
    potential_baryons = reserve_baryons(baryon_list).sort(key = mass)
    potential_leptons = reserve_leptons(lepton_list)
    
    # check if neutron and electron chemical potentials are large enough 
    if (threshold(baryon, omega_field, rho_field, sigma_field, neutron_chem_pot, electron_chem_pot)):
        # add potential baryon to current baryon list
        current_baryons.append(potential_baryons[0])
        # remove potential baryon from potential baryon list
        # assuming is already ordered in terms of mass
        potential_baryons.remove(potential_baryons[0])

In [17]:
def mass(baryon):
    return baryon.num_mass

In [26]:
test_baryons = [Proton, Neutron, Lambda, Sigma_neu]

In [28]:
test_baryons.remove(test_baryons[0])
for baryon in test_baryons:
    print(baryon.name)

Lambda
Sigma_0


In [10]:
def reserve_baryons(baryon_list):
    # returns list of baryons that aren't protons or neutrons
    reserve_list = []
    for baryon in baryon_list:
        if (baryon != Proton and baryon != Neutron):
            reserve_list.append(baryon)
    return reverse_list 

def reserve_leptons(lepton_list):
    # return list of leptons that aren't electrons 
    reserve_list = []
    for lepton in lepton_list:
        if (lepton != electron):
            reserve_list.append(lepton)
    return reserve_list 